In [22]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,
KNeighborsClassifier)
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/baileyrusso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/baileyrusso/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [12]:
postmates = pd.read_csv('/Users/baileyrusso/PycharmProjects/GrubHubScraper/postmates_all7.csv')
postmates = postmates.loc[:, ~postmates.columns.str.contains('^Unnamed')]

In [13]:
postmates

,Name,Category,Favorites,Address,MenuItem,MenuItemDescription,MenuItemPrice,Address_Coordinates,Latitude,Longitude
0,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,BL Burrito (D),"Scrambled eggs, bacon, avocado, feta, crispy p...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785
1,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Avocado Smash (D),"Our classic avo smash with feta, heirloom toma...",14.3,"(40.7207158, -73.9947851)",40.720716,-73.994785
2,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Latte,Double shot of our signature maverick espresso...,NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785
3,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Chai Latte,"Chai extract with your choice of milk, poured ...",NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785
4,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Athletes Burrito (D),"Egg whites, sauteed spinach, roasted tomatoes,...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785
...,...,...,...,...,...,...,...,...,...,...
97226,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Ricotta Cheesecake,NaN,16.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287
97227,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Asparagus,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287
97228,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Spinach,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287
97229,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Haricots,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287


In [14]:
postmatesSub = postmates[['Name','MenuItemPrice','Category','MenuItem','MenuItemDescription']]

# Split Menu Item Description By Word

In [15]:
postmatesSub['MenuItemDescription'] = list(map(lambda row: str(row).replace(',','').split(' ') if pd.notnull(row) else [row], postmatesSub['MenuItemDescription']))




<ipython-input-15-7d6af560d296>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postmatesSub['MenuItemDescription'] = list(map(lambda row: str(row).replace(',','').split(' ') if pd.notnull(row) else [row], postmatesSub['MenuItemDescription']))


In [16]:
postmatesSub['MenuItem'] = list(map(lambda row: str(row).split(' ') if pd.notnull(row) else [row], postmatesSub['MenuItem']))


<ipython-input-16-7f7ee7678721>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postmatesSub['MenuItem'] = list(map(lambda row: str(row).split(' ') if pd.notnull(row) else [row], postmatesSub['MenuItem']))


In [17]:
postmatesSub['Category'] = list(map(lambda row: [row] if pd.notnull(row) else [row], postmatesSub['Category']))

<ipython-input-17-48c0dd3d55ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postmatesSub['Category'] = list(map(lambda row: [row] if pd.notnull(row) else [row], postmatesSub['Category']))


In [18]:
#combine columns into bag of words
postmates['BagOfWords'] = postmatesSub['MenuItem'] + postmatesSub['MenuItemDescription']



In [19]:
#lowercase all words
postmates['BagOfWords'] = list(map(lambda row: [str(each_word).lower() for each_word in row], postmates['BagOfWords']))


In [20]:
#remove stopwords
stop_words = stopwords.words('english')
postmates['BagOfWords'] = list(map(lambda word_list: [word for word in word_list if word not in stopwords.words('english')], postmates['BagOfWords']))




In [ ]:
postmates

In [23]:
#lemmatize words
lemmatizer = WordNetLemmatizer()
postmates['BagOfWords'] = list(map(lambda words: [lemmatizer.lemmatize(word) for word in words], postmates['BagOfWords']))




In [27]:
postmates

,Name,Category,Favorites,Address,MenuItem,MenuItemDescription,MenuItemPrice,Address_Coordinates,Latitude,Longitude,BagOfWords
0,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,BL Burrito (D),"Scrambled eggs, bacon, avocado, feta, crispy p...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785,"[bl, burrito, (d), scrambled, egg, bacon, avoc..."
1,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Avocado Smash (D),"Our classic avo smash with feta, heirloom toma...",14.3,"(40.7207158, -73.9947851)",40.720716,-73.994785,"[avocado, smash, (d), classic, avo, smash, fet..."
2,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Latte,Double shot of our signature maverick espresso...,NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785,"[iced, latte, double, shot, signature, maveric..."
3,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Chai Latte,"Chai extract with your choice of milk, poured ...",NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785,"[iced, chai, latte, chai, extract, choice, mil..."
4,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Athletes Burrito (D),"Egg whites, sauteed spinach, roasted tomatoes,...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785,"[athlete, burrito, (d), egg, white, sauteed, s..."
...,...,...,...,...,...,...,...,...,...,...,...
97226,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Ricotta Cheesecake,NaN,16.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,"[ricotta, cheesecake, nan]"
97227,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Asparagus,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,"[asparagus, nan]"
97228,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Spinach,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,"[spinach, nan]"
97229,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Haricots,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,"[haricot, nan]"


In [28]:
postmates['BagOfWords'] = list(map(lambda row: ' '.join(row), postmates['BagOfWords']))

In [29]:
postmates

,Name,Category,Favorites,Address,MenuItem,MenuItemDescription,MenuItemPrice,Address_Coordinates,Latitude,Longitude,BagOfWords
0,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,BL Burrito (D),"Scrambled eggs, bacon, avocado, feta, crispy p...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785,bl burrito (d) scrambled egg bacon avocado fet...
1,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Avocado Smash (D),"Our classic avo smash with feta, heirloom toma...",14.3,"(40.7207158, -73.9947851)",40.720716,-73.994785,avocado smash (d) classic avo smash feta heirl...
2,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Latte,Double shot of our signature maverick espresso...,NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785,iced latte double shot signature maverick espr...
3,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Chai Latte,"Chai extract with your choice of milk, poured ...",NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785,iced chai latte chai extract choice milk poure...
4,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Athletes Burrito (D),"Egg whites, sauteed spinach, roasted tomatoes,...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785,athlete burrito (d) egg white sauteed spinach ...
...,...,...,...,...,...,...,...,...,...,...,...
97226,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Ricotta Cheesecake,NaN,16.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,ricotta cheesecake nan
97227,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Asparagus,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,asparagus nan
97228,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Spinach,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,spinach nan
97229,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Haricots,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,haricot nan


In [30]:
count = CountVectorizer()
countVec = count.fit(postmates['BagOfWords'])
count_matrix = countVec.transform(postmates['BagOfWords'])

In [38]:
def getMostSimilarItems(user_input):
    user_input = user_input.split(' ')
    user_input = [word for word in user_input if word not in stopwords.words('english')]
    user_input = [lemmatizer.lemmatize(word) for word in user_input]
    user_input = ' '.join(user_input)
    user_input = [user_input]
    
    count_matrix_inp = countVec.transform(user_input)
    
    cosine_sim = cosine_similarity(count_matrix, count_matrix_inp)
    most_similar = cosine_sim[:,0].argsort(axis=0)[::-1]

    similar = postmates.iloc[most_similar][['Name','Category','MenuItem','Favorites']]
    
    return pd.DataFrame(similar)


In [39]:
most_similar = getMostSimilarItems('Chicken parm')
most_similar.head(25)

,Name,Category,MenuItem,Favorites
81427,Fumo,PIZZA,Chicken Parm Hero,80.0
94334,Peperino NYC Pizza,PIZZA,Chicken Parm/Spaghetti,0.0
61307,Marinara Pizza,PIZZA,Chicken Parm Hero,55.0
61265,Marinara Pizza,PIZZA,Chicken Parm Slice,55.0
76550,Marinara Pizza,ITALIAN,Chicken Parm Slice,76.0
76418,Little Italy Pizza,PIZZA,Chicken Parm Hero,0.0
64681,Librettos Pizzeria,PIZZA,Chicken Parm Hero,54.0
12003,Parm,ITALIAN,Chicken Parm Hero,535.0
69910,Parm,ITALIAN,Chicken Parm Hero,411.0
12013,Parm,ITALIAN,Chicken Parm Roll,535.0


In [46]:
postmates['BagOfWords'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
97226    False
97227    False
97228    False
97229    False
97230    False
Name: BagOfWords, Length: 97231, dtype: bool

In [40]:
postmates.to_csv('postmates_BOW.csv')

In [41]:
postmates

,Name,Category,Favorites,Address,MenuItem,MenuItemDescription,MenuItemPrice,Address_Coordinates,Latitude,Longitude,BagOfWords
0,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,BL Burrito (D),"Scrambled eggs, bacon, avocado, feta, crispy p...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785,bl burrito (d) scrambled egg bacon avocado fet...
1,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Avocado Smash (D),"Our classic avo smash with feta, heirloom toma...",14.3,"(40.7207158, -73.9947851)",40.720716,-73.994785,avocado smash (d) classic avo smash feta heirl...
2,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Latte,Double shot of our signature maverick espresso...,NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785,iced latte double shot signature maverick espr...
3,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Chai Latte,"Chai extract with your choice of milk, poured ...",NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785,iced chai latte chai extract choice milk poure...
4,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Athletes Burrito (D),"Egg whites, sauteed spinach, roasted tomatoes,...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785,athlete burrito (d) egg white sauteed spinach ...
...,...,...,...,...,...,...,...,...,...,...,...
97226,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Ricotta Cheesecake,NaN,16.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,ricotta cheesecake nan
97227,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Asparagus,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,asparagus nan
97228,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Spinach,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,spinach nan
97229,Amaranth,MEDITERRANEAN,0.0,21 E 62ND ST,Haricots,NaN,12.0,"(40.765803899999995, -73.97028721319174)",40.765804,-73.970287,haricot nan
